In [ ]:
print("test")

<a href="https://colab.research.google.com/github/hatthebutterfly/Translator-seq2seq/blob/main/translation_model_of_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 參考連結
1. [淺談神經機器翻譯& 用Transformer 與TensorFlow 2 英翻中](https://leemeng.tw/neural-machine-translation-with-transformer-and-tensorflow2.html)
2. [Tensorflow 官方文檔](https://www.tensorflow.org/text/tutorials/nmt_with_attention)

將需要的模組 import 出來

In [ ]:
import os
import re
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from pprint import pprint
from google.colab import drive
from IPython.display import clear_output

### 資料集選取
WMT19 資料集中有許多類型的資料集，如下所示。
這裡我們將使用**新聞評論**作為訓練資料。

In [ ]:
tmp_builder = tfds.builder("wmt19_translate/zh-en")
pprint(tmp_builder.subsets)

'''
**表示為文本資料集
[
   'newscommentary_v14',->新聞評論**
   'wikititles_v1',->維基百科**
   'uncorpus_v1',->聯合國**
   'casia2015',
   'casict2011',
   'casict2015',
   'datum2015',
   'datum2017',
   'neu2017'->表面缺陷資料集
],'''

{Split('train'): ['newscommentary_v14',
                  'wikititles_v1',
                  'uncorpus_v1',
                  'casia2015',
                  'casict2011',
                  'casict2015',
                  'datum2015',
                  'datum2017',
                  'neu2017'],
 Split('validation'): ['newstest2018']}


"\n**表示為文本資料集\n[\n   'newscommentary_v14',->新聞評論**\n   'wikititles_v1',->維基百科**\n   'uncorpus_v1',->聯合國**\n   'casia2015',\n   'casict2011',\n   'casict2015',\n   'datum2015',\n   'datum2017',\n   'neu2017'->表面缺陷資料集\n],"

In [ ]:
drive.mount('/content/drive')
download_dir=os.chdir('/content/drive/My Drive/download_data') #自己的路徑
config = tfds.translate.wmt.WmtConfig(
    version=tfds.core.Version('0.0.3'),
    language_pair=("zh", "en"),
    subsets={
        tfds.Split.TRAIN: ["newscommentary_v14"]
    },
)
builder = tfds.builder("wmt_translate", config=config)
builder.download_and_prepare(download_dir=download_dir)
#155778筆資料

Mounted at /content/drive


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/wmt_translate/zh-en/0.0.3.incompleteFAHOQX/wmt_translate-train.tfrecord*..…

Dataset wmt_translate downloaded and prepared to /root/tensorflow_datasets/wmt_translate/zh-en/0.0.3. Subsequent calls will reuse this data.


### 分割資料集
由於整個資料及共有10萬以上的文本，我只取了大約 30% 的資料來訓練 ( 約 40000 筆資料 )

In [ ]:
#load data and split into validation_data and train_data
print(builder.info)
data=builder.as_dataset(split=['train[:30%]','train[99%:100%]']) #處理完後有 44627+2 筆資料
data1=builder.as_dataset(split=['train[30%:60%]','train[98%:99%]'])
data_corpus=builder.as_dataset(split=['train[:60%]','train[98%:99%]'])
train_ex,val_ex=data
train_for_corpus,val_for_corpus=data_corpus
print(train_ex)
print(val_ex)

ds=list(train_ex)
print(len(ds))

tfds.core.DatasetInfo(
    name='wmt_translate',
    full_name='wmt_translate/zh-en/0.0.3',
    description="""
    Translate dataset based on the data from statmt.org.
    
    Versions exists for the different years using a combination of multiple data
    sources. The base `wmt_translate` allows you to create your own config to choose
    your own data/language pair by creating a custom `tfds.translate.wmt.WmtConfig`.
    
    ```
    config = tfds.translate.wmt.WmtConfig(
        version="0.0.1",
        language_pair=("fr", "de"),
        subsets={
            tfds.Split.TRAIN: ["commoncrawl_frde"],
            tfds.Split.VALIDATION: ["euelections_dev2019"],
        },
    )
    builder = tfds.builder("wmt_translate", config=config)
    ```
    """,
    homepage='http://www.statmt.org',
    data_path='/root/tensorflow_datasets/wmt_translate/zh-en/0.0.3',
    file_format=tfrecord,
    download_size=35.00 MiB,
    dataset_size=87.13 MiB,
    features=Translation({
        'en': Text

### 建立英文語料庫
  如下所示，並在建立好之後存入雲端，下一次就不用浪費時間再建一次。
  
  另外，我在用&nbsp;*tfds.builder()*&nbsp;建資料時發現 data 中的 **"en"索引對應的是中文、"zh"索引對應的是英文**。目前找不出問題所在，所以程式中會直接進行互換。



In [ ]:
%%time
vocab_corpus_dir=r'/content/drive/My Drive/en_vocab_2'
try:
  subword_encoder_en = tfds.deprecated.text.SubwordTextEncoder.load_from_file(vocab_corpus_dir)
  print(f"載入已建立的字典： {vocab_corpus_dir}")
except:
  print("沒有已建立的字典，從頭建立。")
  subword_encoder_en =  tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
      (ex['zh'].numpy() for ex in train_for_corpus),
      target_vocab_size=2**13) # 有需要可以調整字典大小

  # 將字典檔案存下以方便下次 warmstart
  subword_encoder_en.save_to_file(vocab_corpus_dir)


print(f"字典大小：{subword_encoder_en.vocab_size}")
print(f"前 10 個 subwords：{subword_encoder_en.subwords[:10]}")
print()

載入已建立的字典： /content/drive/My Drive/en_vocab_2
字典大小：8165
前 10 個 subwords：[', ', 'the_', 'to_', 'of_', 'and_', 's_', 'in_', 'a_', 'is_', 'that_']

CPU times: user 95.1 ms, sys: 5.67 ms, total: 101 ms
Wall time: 1.04 s


### 建立中文語料庫
同上

In [ ]:
%%time
zh_vocab_corpus_dir=r'/content/drive/My Drive/zh_vocab_2'
try:
  subword_encoder_zh = tfds.deprecated.text.SubwordTextEncoder.load_from_file(zh_vocab_corpus_dir)
  print(f"載入已建立的字典： {zh_vocab_corpus_dir}")
except:
  print("沒有已建立的字典，從頭建立。")
  subword_encoder_zh =  tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
      (ex['en'].numpy() for ex in train_for_corpus),
      target_vocab_size=2**13,max_subword_length=1) # 有需要可以調整字典大小

  # 將字典檔案存下以方便下次 warmstart
  subword_encoder_zh.save_to_file(zh_vocab_corpus_dir)


print(f"字典大小：{subword_encoder_zh.vocab_size}")
print(f"前 10 個 subwords：{subword_encoder_zh.subwords[:10]}")
print()

載入已建立的字典： /content/drive/My Drive/zh_vocab_2
字典大小：4683
前 10 個 subwords：['的', '，', '。', '国', '在', '是', '一', '和', '不', '这']

CPU times: user 60.2 ms, sys: 5.06 ms, total: 65.2 ms
Wall time: 749 ms


上述的語料庫可以直接幫我們把輸入的文字轉成索引，十分方便

In [ ]:
ex_text='Encoding is fully invertible because all out-of-vocab wordpieces are byte-encoded. --Johnson'
ind=subword_encoder_en.encode(ex_text)
print(ind)

[1114, 425, 505, 9, 1033, 3120, 1955, 3423, 124, 70, 368, 7954, 289, 7954, 5119, 1366, 3729, 6035, 6, 19, 1513, 670, 7954, 5385, 2339, 132, 7954, 7954, 2019, 4402, 2011]


這欄可以看到，當出現語料庫中找不到的字時，他會將字切成更小的部分分別轉換。因此不會怕出現字典裡沒有出現過的字

In [ ]:
for i in ind:
  print('index:',i,' ','original word:',subword_encoder_en.decode([i]))

index: 1114   original word: En
index: 425   original word: co
index: 505   original word: ding 
index: 9   original word: is 
index: 1033   original word: fully 
index: 3120   original word: inv
index: 1955   original word: ert
index: 3423   original word: ible 
index: 124   original word: because 
index: 70   original word: all 
index: 368   original word: out
index: 7954   original word: -
index: 289   original word: of
index: 7954   original word: -
index: 5119   original word: voca
index: 1366   original word: b 
index: 3729   original word: word
index: 6035   original word: piece
index: 6   original word: s 
index: 19   original word: are 
index: 1513   original word: by
index: 670   original word: te
index: 7954   original word: -
index: 5385   original word: enco
index: 2339   original word: ded
index: 132   original word: . 
index: 7954   original word: -
index: 7954   original word: -
index: 2019   original word: Jo
index: 4402   original word: hn
index: 2011   original word:

In [ ]:
en = "The eurozone’s collapse forces a major realignment of European politics."
zh = "欧元区的瓦解强迫欧洲政治进行一次重大改组。"

# 將文字轉成為 subword indices
en_indices = subword_encoder_en.encode(en)
zh_indices = subword_encoder_zh.encode(zh)

print("[英中原文]（轉換前）")
print(en)
print(zh)
print()
print('-' * 20)
print()
print("[英中序列]（轉換後）")
print(en_indices)
print(zh_indices)

[英中原文]（轉換前）
The eurozone’s collapse forces a major realignment of European politics.
欧元区的瓦解强迫欧洲政治进行一次重大改组。

--------------------

[英中序列]（轉換後）
[16, 938, 11, 6, 1667, 772, 8, 219, 2162, 2833, 239, 4, 87, 1222, 7955]
[44, 201, 177, 1, 865, 204, 234, 600, 44, 91, 17, 126, 100, 37, 7, 279, 85, 18, 211, 268, 3]


### 文字的接續處理

In [ ]:
def encode(en_t,zh_t):
  #添加 BOS token && EOS token的函數
  # 我們可以使用 subword_encoder_en.vocab_size 這個值作為 BOS 的索引值
  # 用 subword_encoder_en.vocab_size + 1 作為 EOS 的索引值
  #en_t=dic['zh']
  print(en_t)
  print(en_t.numpy().decode('utf-8'))
  #zh_t=dic['en']
  zh_indices=[subword_encoder_zh.vocab_size]+subword_encoder_zh.encode(zh_t.numpy())+[subword_encoder_zh.vocab_size+1]
  en_indices=[subword_encoder_en.vocab_size]+subword_encoder_en.encode(en_t.numpy())+[subword_encoder_en.vocab_size+1]

  return en_indices,zh_indices

def tf_encode(en_t,zh_t):
  # 在 `tf_encode` 函式裡頭的 `en_t` 與 `zh_t` 都不是 Eager Tensors
  # 要到 `tf.py_funtion` 裡頭才是
  # 另外因為索引都是整數，所以使用 `tf.int64`
  return tf.py_function(encode,inp=[en_t,zh_t],Tout=[tf.int64,tf.int64])

In [ ]:
for ex in train_ex:
  en_t=ex['zh']
  zh_t=ex['en']
  break

#en_ind=encode_en(en_t)
#print(en_ind)
#print(zh_ind)


In [ ]:
train_dataset=train_ex.map(lambda x:tf_encode(x['zh'],x['en']))


In [ ]:
print(train_dataset)
en_ind,zh_ind=next(iter(train_dataset))
print(en_ind)
print(zh_ind)

<_MapDataset element_spec=(TensorSpec(shape=<unknown>, dtype=tf.int64, name=None), TensorSpec(shape=<unknown>, dtype=tf.int64, name=None))>
tf.Tensor(b'The fear is real and visceral, and politicians ignore it at their peril.', shape=(), dtype=string)
The fear is real and visceral, and politicians ignore it at their peril.
tf.Tensor(
[8165   16 1217    9  249    5 1213 2237  173    1    5 1030 5774   21
   38   31 3561 8017 7955 8166], shape=(20,), dtype=int64)
tf.Tensor(
[4683   10  150  572 1249    6  386   53   29  193    5    1    3 4459
  953  438  125    1   17  126   33   20   98 1110 1245  861    3 4684], shape=(28,), dtype=int64)


### 去除過長的資料
超過 40 個字的句子會被去掉

In [ ]:
sen_max_len=40
def sentence_length_filter(en,zh,max_len=sen_max_len):
  return tf.logical_and(tf.size(en)<=max_len,tf.size(zh)<=max_len)
train_dataset=train_dataset.filter(sentence_length_filter)

In [ ]:
#建立正式資料集
buffer_size=15000
train_dataset=(train_ex
        .map(lambda x:tf_encode(x['zh'],x['en']),tf.data.AUTOTUNE)
        .filter(sentence_length_filter)
        .cache() #加快讀取
        .shuffle(buffer_size) #打亂
        .padded_batch(64,padded_shapes=([-1],[-1]))
        .prefetch(tf.data.experimental.AUTOTUNE) #加速
              )
val_dataset=(val_ex
        .map(lambda x:tf_encode(x['zh'],x['en']),tf.data.AUTOTUNE)
        .filter(sentence_length_filter)
        .cache() #加快讀取
        .shuffle(buffer_size) #打亂
        .padded_batch(64,padded_shapes=([-1],[-1]))
        .prefetch(tf.data.experimental.AUTOTUNE) #加速
              )

In [ ]:
print(train_dataset)
clear_output(wait=True)


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None), dtype=tf.int64, name=None), TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>


In [ ]:
#測試一下怎麼轉成張量
demo_examples = [
    ("It is important.", "这很重要。"),
    ("The numbers speak for themselves.", "数字证明了一切。"),
]

demo_examples_tensor=tf.data.Dataset.from_tensor_slices(([en for en,_ in demo_examples],[zh for _,zh in demo_examples])) #注意：for迴圈中的 _ 代表"跳過第二個值，我不在乎他是誰"
demo_examples_tensor=demo_examples_tensor.map(tf_encode).padded_batch(2,padded_shapes=([-1],[-1]))
inp,tar=next(iter(demo_examples_tensor))
print(inp.shape)
print(f'inp:{inp}')
print(f'tar:{tar}')

tf.Tensor(b'It is important.', shape=(), dtype=string)
It is important.
tf.Tensor(b'The numbers speak for themselves.', shape=(), dtype=string)
The numbers speak for themselves.
(2, 8)
inp:[[8165  102    9 1025 7955 8166    0    0]
 [8165   16 4128 5485   12 2476 7955 8166]]
tar:[[4683   10  236   85   27    3 4684    0    0    0]
 [4683  167  491  406  190   14    7  549    3 4684]]


In [ ]:
# build encoder
# 框架來自 https://keras.io/zh/layers/writing-your-own-keras-layers/ 和 https://www.tensorflow.org/text/tutorials/nmt_with_attention#the_encoder
# 解釋：https://sparrow.dev/keras-custom-layer/
class Encoder(tf.keras.layers.Layer):
  def __init__(self,units):
    super().__init__() # 繼承 tf.keras.layers.Layer 中的方法 (如下面的 call() 或是 build() 之類的。也可以去翻source code：https://github.com/keras-team/keras/blob/master/keras/engine/base_layer.py)
    self.units=256
    self.vocab_size=subword_encoder_en.vocab_size+2

    self.embedding=tf.keras.layers.Embedding(input_dim=self.vocab_size,output_dim=self.units,mask_zero=True) # mask；由於前面做了 padding，需要mask 把牠們遮起來(padding的部分)

    self.rnn=tf.keras.layers.Bidirectional(merge_mode='sum',layer=tf.keras.layers.GRU(self.units,return_sequences=True,return_state=True,recurrent_initializer='glorot_uniform')) #sum 的原因是attention layer會取平均

  def call(self,x):
    x=self.embedding(x)
    #print(x)
    x=self.rnn(x)
    return x
  def convert_input(self,texts):
    texts = tf.convert_to_tensor(texts)
    if len(texts.shape) == 0:
      texts = tf.convert_to_tensor(texts)[tf.newaxis]
    return texts



In [ ]:
#ex_inp,ex_tar=next(iter(train_dataset))
encoder=Encoder(256)
ex_content=encoder(inp)
print(inp.shape)
for i in range(len(ex_content)):
  print(ex_content[i].shape)


(2, 8)
(2, 8, 256)
(2, 256)
(2, 256)


In [ ]:
print(len(ex_content))
print(ex_content[0])

3
tf.Tensor(
[[[-0.012569    0.00996978 -0.0359096  ...  0.00353702 -0.01153903
   -0.00665021]
  [ 0.00914672 -0.0020915  -0.03005479 ... -0.00529666  0.00694257
    0.00672482]
  [-0.02034006  0.01507995 -0.00585481 ... -0.02975649 -0.00027967
    0.02432467]
  ...
  [ 0.00066443  0.02594291 -0.00710115 ...  0.00886957 -0.00983894
   -0.00194608]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]]

 [[-0.01159634  0.01014251 -0.02694752 ...  0.01236947 -0.00137768
   -0.00423439]
  [ 0.00575708  0.0003239  -0.02055835 ...  0.02920941  0.03188482
   -0.00222713]
  [ 0.01952353  0.00103221 -0.00823582 ... -0.00129587  0.02462072
    0.00871017]
  ...
  [-0.01950962  0.01665722 -0.00477417 ...  0.02567497 -0.00095139
   -0.01003937]
  [-0.02130001  0.02151782 -0.01477064 ...  0.03142406  0.00404314
   -0.01476767]
  [-0.00817252  0.02834298 -0.00875838 ...  0.01645571 -0.01504016
   -0.008

In [ ]:
# building attention layer
# model structure：https://www.tensorflow.org/text/tutorials/nmt_with_attention#the_attention_layer
# explaining query、key and value ：https://stats.stackexchange.com/questions/421935/what-exactly-are-keys-queries-and-values-in-attention-mechanisms 、https://blog.csdn.net/yangyehuisw/article/details/116207892
# what is attention??：https://leemeng.tw/neural-machine-translation-with-transformer-and-tensorflow2.html && https://www.youtube.com/watch?v=ugWDIIOHtPA
# original paper：https://arxiv.org/abs/1508.04025v5 、https://arxiv.org/abs/1706.03762

class Attention_layer(tf.keras.layers.Layer):
  def __init__(self,units):
    super().__init__()
    self.multi_head=tf.keras.layers.MultiHeadAttention(key_dim=units, num_heads=1) # 也可換成一般attention layer 試看看
    self.layernorm=tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.add=tf.keras.layers.Add()
  def call(self,target,context):
    attn_output,attn_score=self.multi_head(query=target,value=context,return_attention_scores=True)
    attn_scores=tf.reduce_mean(attn_score,axis=1)
    self.last_attention_weight=attn_scores

    target=self.add([target,attn_output])
    target=self.layernorm(target)

    return target

In [ ]:
attention_layer=Attention_layer(256)
embed= tf.keras.layers.Embedding(subword_encoder_zh.vocab_size+2,output_dim=256, mask_zero=True)
ex_tar=embed(tar)
print(ex_tar)
result=attention_layer(ex_tar,ex_content[0])
print(result)
print()
print("shape of attention_weight",attention_layer.last_attention_weight.shape)

tf.Tensor(
[[[-0.03955146 -0.03485359  0.03563461 ...  0.04266138 -0.01980332
   -0.03396132]
  [-0.03167723 -0.03576545 -0.04727941 ...  0.01577756 -0.03408135
   -0.02617952]
  [-0.03596318  0.03088248 -0.0113495  ... -0.01786476 -0.01007621
   -0.00193285]
  ...
  [-0.01103805  0.0304517  -0.01996478 ... -0.02361752 -0.02935146
   -0.04219384]
  [-0.01103805  0.0304517  -0.01996478 ... -0.02361752 -0.02935146
   -0.04219384]
  [-0.01103805  0.0304517  -0.01996478 ... -0.02361752 -0.02935146
   -0.04219384]]

 [[-0.03955146 -0.03485359  0.03563461 ...  0.04266138 -0.01980332
   -0.03396132]
  [-0.00863092 -0.04799482  0.00204455 ...  0.01897422  0.01390536
   -0.02410041]
  [ 0.04318872  0.0194147   0.02690503 ... -0.03652961  0.02635684
    0.02104386]
  ...
  [-0.02733526  0.005688   -0.00082359 ...  0.00041948  0.03968402
   -0.00374059]
  [-0.00446244 -0.02572393 -0.03754602 ... -0.0275334  -0.01720031
   -0.01777123]
  [ 0.000543    0.02043648 -0.03354366 ...  0.02903027  0.0402

In [ ]:
# Decoder

class Decoder(tf.keras.layers.Layer):
  @classmethod #定義 method 以後直接叫
  def add_method(cls,function):
    setattr(cls,function.__name__,function)
    return function
  def __init__(self,units=256):
    super(Decoder,self).__init__()
    self.vocab_size_zh=subword_encoder_zh.vocab_size+2
    self.units=units

    self.start_token=subword_encoder_zh.vocab_size
    self.end_token=subword_encoder_zh.vocab_size+1

    # embed the target sentences
    self.embedding=tf.keras.layers.Embedding(input_dim=self.vocab_size_zh,output_dim=self.units,mask_zero=True)

    self.rnn=tf.keras.layers.GRU(units,return_sequences=True,return_state=True,recurrent_initializer='glorot_uniform')

    self.attention=Attention_layer(self.units)

    # output layer outputs the logits of each words
    self.output_layer=tf.keras.layers.Dense(self.vocab_size_zh)

  def call(self, enc_input_context, target_inp, state=None, return_state=False):

    target_inp=self.embedding(target_inp)

    target_inp,state=self.rnn(target_inp,initial_state=state)

    target_inp=self.attention(target_inp,enc_input_context)
    self.last_attention_weight=self.attention.last_attention_weight

    logits=self.output_layer(target_inp)

    if return_state:
      return logits,state
    else :
      return logits

  def get_initial_state(self,context):
    batch_size=tf.shape(context)[0]
    start_tokens=tf.fill([batch_size,1],self.start_token)
    done=tf.zeros([batch_size,1],dtype=tf.bool)
    embedded=self.embedding(start_tokens)
    return start_tokens,done,self.rnn.get_initial_state(embedded)[0]


  def get_next_token(self,context,token,done,state,temperature=0.0):
    logits, state = self(context, token,state = state,return_state=True)

    if temperature==0.0:
      next_token=tf.math.argmax(logits,axis=-1)
    else :
      logits=logits[:,-1,:]/temperature # temperature 是用來 control 數據的大小。藉此影響機率分布 (簡單來說，temperature值愈大，會讓機率分布平滑；愈小，會讓機率分布愈尖)
      next_token=tf.random.categorical(logits, num_samples=1) # 從機率分布隨機取樣

    # If a sequence produces an `end_token`, set it `done`
    done=done | (next_token==self.end_token)

    next_token = tf.where(done, tf.constant(0, dtype=tf.int64), next_token)

    return next_token,done,state




In [ ]:
decoder=Decoder()
logits=decoder(ex_content[0],tar)
print(logits)

tf.Tensor(
[[[ 0.26220107  0.4786552   0.05658217 ... -0.13869561  0.0703841
   -0.3080092 ]
  [ 0.19234364  0.39938006  0.25272167 ... -0.34182122  0.6408793
    0.11927228]
  [ 0.25683275  0.5627353   0.2892809  ... -0.32869914  0.66460747
   -0.12497947]
  ...
  [ 0.06705103 -0.00463379 -0.00141484 ... -0.16923645  0.12399185
    0.21130039]
  [ 0.06705103 -0.00463379 -0.00141484 ... -0.16923645  0.12399185
    0.21130039]
  [ 0.06705103 -0.00463379 -0.00141484 ... -0.16923645  0.12399185
    0.21130039]]

 [[ 0.19974394  0.45650187  0.0143076  ... -0.19087085  0.04297695
   -0.34286934]
  [ 0.0469609   0.52059597 -0.20718823 ... -0.26249734 -0.14952412
    0.28368154]
  [-0.58126223  0.57021385 -0.01349556 ... -0.5656019   0.45218128
    0.37520894]
  ...
  [ 0.14995575  0.8210808   0.19149473 ... -0.75412977 -0.2823381
    0.95478827]
  [-0.07680836  0.9183817  -0.21442333 ... -0.30494145 -0.34317774
    0.7095268 ]
  [ 0.15428549  0.5086141  -0.01836307 ...  0.09254748 -0.1524793

In [ ]:
print(f"Original sentence：{inp}")
next_token,done,state=decoder.get_initial_state(ex_content[0])
tokens=[]

for i in range(10):
  next_token,done,state=decoder.get_next_token(ex_content[0],next_token,done,state,1)

  tokens.append(next_token)
print(tokens)
tokens=tf.concat(tokens,axis=-1)
tokens=tokens.numpy().tolist()
print(tokens)
for i in range(len(tokens)):
  words=subword_encoder_zh.decode(tokens[i])
  result=tf.strings.reduce_join(words)
  print(result.numpy().decode('utf-8'))
# 因為還沒訓練，跑出來的中文沒有意義，只是隨機取樣的結果

Original sentence：[[8165  102    9 1025 7955 8166    0    0]
 [8165   16 4128 5485   12 2476 7955 8166]]
[<tf.Tensor: shape=(2, 1), dtype=int64, numpy=
array([[1109],
       [4209]])>, <tf.Tensor: shape=(2, 1), dtype=int64, numpy=
array([[ 702],
       [3479]])>, <tf.Tensor: shape=(2, 1), dtype=int64, numpy=
array([[4252],
       [3669]])>, <tf.Tensor: shape=(2, 1), dtype=int64, numpy=
array([[1560],
       [4437]])>, <tf.Tensor: shape=(2, 1), dtype=int64, numpy=
array([[ 930],
       [2833]])>, <tf.Tensor: shape=(2, 1), dtype=int64, numpy=
array([[1769],
       [3649]])>, <tf.Tensor: shape=(2, 1), dtype=int64, numpy=
array([[3908],
       [1216]])>, <tf.Tensor: shape=(2, 1), dtype=int64, numpy=
array([[4676],
       [1266]])>, <tf.Tensor: shape=(2, 1), dtype=int64, numpy=
array([[1010],
       [ 375]])>, <tf.Tensor: shape=(2, 1), dtype=int64, numpy=
array([[2435],
       [3785]])>]
[[1109, 702, 4252, 1560, 930, 1769, 3908, 4676, 1010, 2435], [4209, 3479, 3669, 4437, 2833, 3649, 1216, 

In [ ]:
class Translator(tf.keras.Model):
  def __init__(self,units):
    super().__init__()
    encoder=Encoder(units)
    decoder=Decoder(units)

    self.encoder=encoder
    self.decoder=decoder

  def call(self,context,target):
    #print(type(inputs))
    #context,target=inputs
    context=self.encoder(context)
    logits=self.decoder(context[0],target)

      #TODO(b/250038731): remove this

      # 目前不知道為何要刪除，之後有解答時在更新
    try:
        # Delete the keras mask, so keras doesn't scale the loss+accuracy.
      del logits._keras_mask
    except AttributeError:
      pass

    return logits


In [ ]:
def masked_loss(y_true,y_pred):
  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,reduction='none')
  loss=loss_fn(y_true,y_pred)

  mask=tf.cast(y_true!=0,dtype=loss.dtype)
  loss*=mask
  return tf.reduce_mean(loss) #文本長度不盡相同，因此需 normalization

def masked_acc(y_true, y_pred):
  # Calculate the loss for each item in the batch.
  y_pred = tf.argmax(y_pred, axis=-1) # 取一個軸中最大值的 index (最大代表機率最大)
  y_pred = tf.cast(y_pred, y_true.dtype)

  match = tf.cast(y_true == y_pred, tf.float32)
  mask = tf.cast(y_true != 0, tf.float32)

  return tf.reduce_sum(match)/tf.reduce_sum(mask)
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='train_accuracy')

In [ ]:
real = tf.constant([1, 1, 0], shape=(1, 3), dtype=tf.float32)
pred = tf.constant([[0, 1], [0, 1], [0, 1]], dtype=tf.float32)
print(masked_loss(real,pred))

tf.Tensor(0.20884116, shape=(), dtype=float32)


In [ ]:
model=Translator(256)

logits = model(inp, tar)
print(logits)
model.compile(optimizer='adam',
              loss=masked_loss,
              metrics=[masked_acc, masked_loss])
optimizers = tf.keras.optimizers.Adam()

tf.Tensor(
[[[-0.3997573   0.22985853 -0.34563777 ... -0.218378   -0.4147678
    0.17566156]
  [-0.3983437  -0.2369987   0.35856488 ... -0.27731246 -0.30288923
   -0.20401235]
  [ 0.198215    0.18273087  0.42200002 ...  0.2723736   0.49920782
    0.23935194]
  ...
  [-0.23560672  0.12494224 -0.11374722 ... -0.19039136  0.06670847
   -0.05699633]
  [-0.23560672  0.12494224 -0.11374722 ... -0.19039136  0.06670847
   -0.05699633]
  [-0.23560672  0.12494224 -0.11374722 ... -0.19039136  0.06670847
   -0.05699633]]

 [[-0.36735564  0.22103739 -0.321472   ... -0.20116776 -0.41373113
    0.14958659]
  [-0.39988726  0.15113534 -0.24203514 ... -0.07360122 -0.04336578
    0.3070155 ]
  [ 0.19507134  0.11394552 -0.2566413  ...  0.10139238 -0.03416552
   -0.12688106]
  ...
  [ 0.38143608 -0.10307219 -0.30642617 ... -0.37838507 -0.6244611
   -0.2371382 ]
  [ 0.39582667  0.1702821   0.2690374  ... -0.35427707 -0.3023736
   -0.00489966]
  [ 0.4624551   0.56259155 -0.21018933 ... -0.24389815 -0.3036613

In [ ]:
checkpoint_path ='/content/drive/My Drive/model'
checkpoint=tf.train.Checkpoint(model)
ckpt_manager = tf.train.CheckpointManager(checkpoint, checkpoint_path, max_to_keep=5)
if ckpt_manager.latest_checkpoint:
  checkpoint.restore(ckpt_manager.latest_checkpoint)

  # 用來確認之前訓練多少 epochs 了
  last_epoch = int(ckpt_manager.latest_checkpoint.split("-")[-1])
  print(f'已讀取最新的 checkpoint，模型已訓練 {last_epoch} epochs。')
else:
  last_epoch = 0
  print("沒找到 checkpoint，從頭訓練。")

已讀取最新的 checkpoint，模型已訓練 3 epochs。


In [ ]:
losses=[]
accuracy=[]
@tf.function  # 讓 TensorFlow 幫我們將 eager code 優化並加快運算
def train_step(inp, tar,epochs):
  # 前面說過的，用去尾的原始序列去預測下一個字的序列
  tar_inp = tar[:, :-1]
  tar_real = tar[:, 1:]


  # 紀錄 Transformer 的所有運算過程以方便之後做梯度下降
  with tf.GradientTape() as tape:
    # 注意是丟入 `tar_inp` 而非 `tar`。記得將 `training` 參數設定為 True
    predictions = model(inp,tar_inp)
    # 跟影片中顯示的相同，計算左移一個字的序列跟模型預測分佈之間的差異，當作 loss
    loss = masked_loss(tar_real, predictions)
    acc=masked_acc(tar_real,predictions)
    #accuracy.append(acc)
    #losses.append(loss)
    print(f'epochs:{epochs}, Loss:{loss}, acc:{acc}')



  # 取出梯度並呼叫前面定義的 Adam optimizer 幫我們更新 Transformer 裡頭可訓練的參數
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizers.apply_gradients(zip(gradients, model.trainable_variables))
  train_loss(loss)
  train_accuracy(tar_real, predictions)



In [ ]:
for epoch in range(3):
  print("----------------------------------------------------------------------------------------")
  for inp, tar in train_dataset:
    train_step(inp, tar,epoch)
  clear_output()
  if (epoch + 1) % 1 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,ckpt_save_path))

Saving checkpoint for epoch 3 at /content/drive/My Drive/model/ckpt-6


In [ ]:
# 給定一個英文句子，輸出預測的中文索引數字序列以及注意權重 dict
def evaluate(inp_sentence):

  # 準備英文句子前後會加上的 <start>, <end>
  start_token = [subword_encoder_en.vocab_size]
  end_token = [subword_encoder_en.vocab_size + 1]

  # inp_sentence 是字串，我們用 Subword Tokenizer 將其變成子詞的索引序列
  # 並在前後加上 BOS / EOS
  inp_sentence = start_token + subword_encoder_en.encode(inp_sentence) + end_token
  encoder_input = tf.expand_dims(inp_sentence, 0)

  # 跟我們在影片裡看到的一樣，Decoder 在第一個時間點吃進去的輸入
  # 是一個只包含一個中文 <start> token 的序列
  decoder_input = [subword_encoder_zh.vocab_size]
  output = tf.expand_dims(decoder_input, 0)  # 增加 batch 維度

  # auto-regressive，一次生成一個中文字並將預測加到輸入再度餵進 Transformer
  for i in range(40):


    # predictions.shape == (batch_size, seq_len, vocab_size)
    predictions= model(encoder_input, output)



    # 將序列中最後一個 distribution 取出，並將裡頭值最大的當作模型最新的預測字
    predictions = predictions[: , -1:, :]  # (batch_size, 1, vocab_size)

    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 遇到 <end> token 就停止回傳，代表模型已經產生完結果
    if tf.equal(predicted_id, subword_encoder_zh.vocab_size + 1):
      return tf.squeeze(output, axis=0)

    #將 Transformer 新預測的中文索引加到輸出序列中，讓 Decoder 可以在產生
    # 下個中文字的時候關注到最新的 `predicted_id`
    output = tf.concat([output, predicted_id], axis=-1)

  # 將 batch 的維度去掉後回傳預測的中文索引序列
  return tf.squeeze(output, axis=0)

In [ ]:
pre=evaluate('China, India, and others have enjoyed continuing economic growth.')
print(pre)
print(tf.strings.reduce_join(subword_encoder_zh.decode(pre[1:])).numpy().decode('utf-8'))

tf.Tensor(
[4683   68   20  208   15   10  127    1  153   27  420  223  539  258
   73  207  280  215  804    3], shape=(20,), dtype=int32)
我们认为这样的需要采取措施就必须立刻。


In [ ]:
# 'We appreciate the need for immediate action.'=> 我们对此必须马上采取行动。
# 'I gently tap Musk on the back in order to remind him that teacher comes.' =>我对于马比利为了宽松政策筹划，我提醒了教师反对师卷入其老师。
# I did absolutely nothing wrong, I never had an affair with him  我曾经绝对错，我绝对不错，我绝对不对他的错误。

